# Error and Messages

```
raise level format;
```

## Level
等级有：
- debug
- log
- notice
- info
- warning
- exception

如果不指定 level，则默认为 exception，并默认停止当前事务。

## Format

`format` 是字符串，可以包含占位符（`%`），占位符的值由后面的参数提供。

In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from D:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


In [2]:
# 例子
sql = """
do $$ 
begin 
  raise info 'information message %', now() ;
  raise log 'log message %', now();
  raise debug 'debug message %', now();
  raise warning 'warning message %', now();
  raise notice 'notice message %', now();
end $$;
"""
cursor.execute(sql)

information message 2024-04-04 12:39:58.641754+08
warning message 2024-04-04 12:39:58.641754+08
notice message 2024-04-04 12:39:58.641754+08


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1c9cd3971c0>

注意，不是所有的等级都会被输出，比如 `info` 和 `log` 会被输出，而 `debug` 不会被输出。这由 client_min_messages 和 log_min_messages 配置参数控制。

## Raising errors

要引发错误，需要在 `raise` 语句后使用 `exception` 级别。还可以用额外的语句增加更多的信息。
```
using option = expression
```

`option` 可以是：
- `hint`
- `detail`
- `errcode`

`expression` 是字符串值表达式。

In [5]:
sql = """
do $$ 
declare
  email varchar(255) := 'info@postgresqltutorial.com';
begin 
  -- check email for duplicate
  -- ...
  -- report duplicate email
  raise exception 'duplicate email: %', email 
		using hint = 'check the email again';
end $$;
"""
cursor.execute(sql)

RaiseException: duplicate email: info@postgresqltutorial.com
HINT:  check the email again
CONTEXT:  在RAISE的第8行的PL/pgSQL函数inline_code_block

# Alert

语法：
```
asset condition [, message];
```

注意：`alert` 最好只用来检查不可能发生的情况，比如检查 `null` 值，而不是用来报告错误，报告错误请使用 `raise`。

`PostgreSQL` 提供了 `plpgsql.check_asserts` 参数来控制 `assert` 语句的行为。如果设置为 `on`，则 `assert` 语句会检查条件是否为 `true`，如果为 `false`，则会引发错误。如果设置为 `off`，则 `assert` 语句不会检查条件，也不会引发错误。

In [7]:
sql = """
do $$
declare 
   film_count integer;
begin
   select count(*)
   into film_count
   from film;
   
   assert film_count > 0, 'Film not found, check the film table';
end$$;
"""
cursor.execute(sql)

InFailedSqlTransaction: 当前事务被终止, 事务块结束之前的查询被忽略

In [2]:
sql = """
do $$
declare 
   film_count integer;
begin
   select count(*)
   into film_count
   from film;
   
   assert film_count > 1000, '1000 Film found, check the film table';
end$$;
"""
cursor.execute(sql)

AssertFailure: 1000 Film found, check the film table
CONTEXT:  在ASSERT的第9行的PL/pgSQL函数inline_code_block